# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.window import Window
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 

In [2]:
spark = SparkSession \
    .builder \
    .appName("Wrangling Data") \
    .getOrCreate()

In [3]:
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

In [4]:
user_log.take(5)

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046'),
 Row(artist='Lily Allen', auth='Logged In', firstName='Elizabeth', gender='F', itemInSession=7, lastName='Chase', length=195.23873, level='free', location='Shreveport-Bossier City, LA', method='PUT', page='NextSong', registration=1512718541284, sessionId=5027, song='Cheryl Tweedy', status=200, ts=1513720878284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1000'),
 Row(artist='Cobra Starship Featuring Leighton Meester', auth='Logged In', firstNa

In [5]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [6]:
user_log.count()

10000

# Question 1

Which page did user id "" (empty string) NOT visit?

In [7]:
# TODO: write your code to answer question 1
user_log.select("page").dropDuplicates().sort("page").show()

+----------------+
|            page|
+----------------+
|           About|
|       Downgrade|
|           Error|
|            Help|
|            Home|
|           Login|
|          Logout|
|        NextSong|
|   Save Settings|
|        Settings|
|Submit Downgrade|
|  Submit Upgrade|
|         Upgrade|
+----------------+



In [8]:
user_log.select("page").where(user_log.userId == "").dropDuplicates().show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



In [9]:
# filter for users with blank user id
blank_pages = user_log.filter(user_log.userId == '') \
    .select(col('page') \
    .alias('blank_pages')) \
    .dropDuplicates()

# get a list of possible pages that could be visited
all_pages = user_log.select('page').dropDuplicates()

# find values in all_pages that are not in blank_pages
# these are the pages that the blank user did not go to
for row in set(all_pages.collect()) - set(blank_pages.collect()):
    print(row.page)

Save Settings
Upgrade
Error
Settings
NextSong
Downgrade
Logout
Submit Downgrade
Submit Upgrade


# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [10]:
# TODO: use this space to explore the behavior of the user with an empty string
user_log.select(["userId", "page", "level", "song"]).where(user_log.userId == "").collect()

[Row(userId='', page='Login', level='free', song=None),
 Row(userId='', page='Home', level='paid', song=None),
 Row(userId='', page='Home', level='free', song=None),
 Row(userId='', page='Home', level='paid', song=None),
 Row(userId='', page='Login', level='paid', song=None),
 Row(userId='', page='Login', level='free', song=None),
 Row(userId='', page='Home', level='paid', song=None),
 Row(userId='', page='Home', level='paid', song=None),
 Row(userId='', page='Login', level='paid', song=None),
 Row(userId='', page='Home', level='paid', song=None),
 Row(userId='', page='Login', level='paid', song=None),
 Row(userId='', page='Home', level='paid', song=None),
 Row(userId='', page='Help', level='paid', song=None),
 Row(userId='', page='Home', level='paid', song=None),
 Row(userId='', page='Login', level='paid', song=None),
 Row(userId='', page='Home', level='free', song=None),
 Row(userId='', page='Login', level='free', song=None),
 Row(userId='', page='Home', level='free', song=None),
 Ro

# Question 3

How many female users do we have in the data set?

In [11]:
# TODO: write your code to answer question 3
user_log_valid = user_log.dropna(how = "any", subset = ["userId", "gender"])
user_log_valid.count()

9664

In [12]:
user_log_valid.select("userId").dropDuplicates().sort("userId").show()

+------+
|userId|
+------+
|    10|
|   100|
|  1000|
|  1003|
|  1005|
|  1006|
|  1017|
|  1019|
|  1020|
|  1022|
|  1025|
|  1030|
|  1035|
|  1037|
|   104|
|  1040|
|  1042|
|  1043|
|  1046|
|  1048|
+------+
only showing top 20 rows



In [13]:
user_log_valid = user_log_valid.filter(user_log_valid["userId"] != "")

In [14]:
user_log_valid.count()

9664

In [15]:
user_log_valid.select("userId").dropDuplicates().sort("userId").where(user_log_valid.gender == "F").count()

462

In [16]:
user_log.filter(user_log.gender == 'F') \
    .select('userId', 'gender') \
    .dropDuplicates() \
    .count()

462

# Question 4

How many songs were played from the most played artist?

In [17]:
# TODO: write your code to answer question 4
user_log.describe("artist").show()

+-------+-----------------+
|summary|           artist|
+-------+-----------------+
|  count|             8347|
|   mean|            461.0|
| stddev|            300.0|
|    min|              !!!|
|    max|ÃÂlafur Arnalds|
+-------+-----------------+



In [18]:
user_log.filter(user_log.page == 'NextSong') \
    .select('Artist') \
    .groupBy('Artist') \
    .agg({'Artist':'count'}) \
    .withColumnRenamed('count(Artist)', 'Artistcount') \
    .sort(desc('Artistcount')) \
    .show(1)

+--------+-----------+
|  Artist|Artistcount|
+--------+-----------+
|Coldplay|         83|
+--------+-----------+
only showing top 1 row



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [22]:
# TODO: write your code to answer question 5
# udf returns a string
function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())

# sort records in reversed time order, and add up the values in the homevisit column for all previous rows
# why desc? we want the total number of songs before homevisit 
user_window = Window \
    .partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

# flag if page is 'Home'
# and calculate the cumulative sum of homevisit by userID in reversed time order
cusum = user_log.filter((user_log.page == 'NextSong') | (user_log.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', Fsum('homevisit').over(user_window))

cusum.select(["userID", "ts", "page", "homevisit", 'period']).where(cusum.userID == "1138").sort("ts").collect()



[Row(userID='1138', ts=1513729066284, page='Home', homevisit=1, period=1),
 Row(userID='1138', ts=1513729066284, page='NextSong', homevisit=0, period=1),
 Row(userID='1138', ts=1513729313284, page='NextSong', homevisit=0, period=1),
 Row(userID='1138', ts=1513729552284, page='NextSong', homevisit=0, period=1),
 Row(userID='1138', ts=1513729783284, page='NextSong', homevisit=0, period=1),
 Row(userID='1138', ts=1513730001284, page='NextSong', homevisit=0, period=1),
 Row(userID='1138', ts=1513730263284, page='NextSong', homevisit=0, period=1),
 Row(userID='1138', ts=1513730518284, page='NextSong', homevisit=0, period=1),
 Row(userID='1138', ts=1513730768284, page='NextSong', homevisit=0, period=1),
 Row(userID='1138', ts=1513731182284, page='NextSong', homevisit=0, period=1),
 Row(userID='1138', ts=1513731435284, page='NextSong', homevisit=0, period=1),
 Row(userID='1138', ts=1513731695284, page='NextSong', homevisit=0, period=1),
 Row(userID='1138', ts=1513731857284, page='NextSong', h

In [23]:
# How many songs do users listen to on average between visiting our home page? 
# count page == 'NextSong' is the total number of songs for each period for each user
cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
|6.9558333333333335|
+------------------+

